In [71]:
import tensorflow as tf
from sklearn.datasets import load_diabetes

X_diab, y_diab = load_diabetes(return_X_y=True)

In [96]:
class Layer:
    def __init__(self, units, input_layer=False):
        self.input_layer = input_layer
        self.units = units
        self.activation = 'relu'

    @tf.function
    def activationFunction(self, z):
        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return tf.maximum(z, tf.zeros(z.shape))

        if self.activation == "sigmoid":
            return 1 / (1 + tf.exp(-z))

    def _weightInit(self, input_size):
        self.w = tf.Variable(tf.random.normal((input_size, self.units), mean = 0, stddev = 1 / input_size, name = 'w'))
        self.bias = tf.Variable(tf.zeros((1, self.units)))

        return self.w, self.bias

    def _update_w(self, grad):
        self.w.assign_sub(0.000001 * grad)

    def _update_b(self, grad):
        self.bias.assign_sub(0.000001 * grad)

    @tf.function
    def __call__(self, X):
        return self.activationFunction(X @ self.w + self.bias)

In [103]:
a = tf.Variable([1, 2, 3, 4, 5])
a.assign_sub([3, 1, 1, 1, 2])

<tf.Variable 'UnreadVariable' shape=(5,) dtype=int32, numpy=array([-2,  1,  2,  3,  3])>

In [97]:
a = tf.Variable(4.)

lst_1 = [4]

lst_1.append(a)

In [ ]:
tf.losses

In [104]:
class DenseNetwork:
    def __init__(self, layers):
        self.layers = layers

        self.tr_vars = []

        for i in range(1, len(self.layers)):
            self.tr_vars.append(self.layers[i]._weightInit(self.layers[i - 1].units))

    def fit(self, X, y):
        X = tf.Variable(X, dtype="float32")
        y = tf.Variable(y, dtype="float32")
        
        for _ in range(100):  
            with tf.GradientTape() as tape:
                loss = self.network(X, y)  

            grads = tape.gradient(loss, self.tr_vars)

            # print(tape.watched_variables())

            for i, (w, b) in enumerate(grads):
                self.layers[i + 1]._update_w(w)
                self.layers[i + 1]._update_b(b)
                
        del tape

    def predict(self, X):
        X = tf.constant(X, dtype='float32')
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    @tf.function
    def network(self, X, y):
        y_pred = self.forward(X)
        loss = tf.losses.mean_squared_error(y_pred, y)
        return loss

    @tf.function
    def forward(self, X):
        for layer in self.layers[1:]:
            X = layer(X)
        return X


In [105]:
nn = DenseNetwork([Layer(10), Layer(30),  Layer(1)])

nn.fit(X_diab, y_diab)

nn.predict(X_diab)

<tf.Tensor: shape=(442, 1), dtype=float32, numpy=
array([[152.27846],
       [151.70099],
       [152.23387],
       [152.3003 ],
       [152.02477],
       [151.63116],
       [151.99591],
       [152.39131],
       [152.30876],
       [152.21037],
       [151.63997],
       [152.41885],
       [151.85573],
       [152.25189],
       [151.81097],
       [152.7271 ],
       [151.8614 ],
       [152.52615],
       [152.05495],
       [151.93608],
       [151.87054],
       [151.91039],
       [151.88632],
       [152.53963],
       [152.00475],
       [152.4521 ],
       [151.54701],
       [151.85281],
       [152.02437],
       [151.82384],
       [151.92508],
       [151.67517],
       [152.46896],
       [151.37433],
       [151.80664],
       [151.8696 ],
       [151.98541],
       [151.902  ],
       [152.64922],
       [152.34448],
       [152.46767],
       [151.69533],
       [152.2816 ],
       [151.58353],
       [152.54901],
       [152.21022],
       [151.92673],
       [15

In [54]:
# import tensorflow.keras.datasets.mnist as tfds
# (x_train, y_train), (x_test, y_test) = tfds.load_data()

In [114]:
v = tf.Variable(1.0)

v.assign_sub(4.0)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=-3.0>

In [155]:
v = tf.Variable(1.0)
c = tf.Variable(2.0)
x = tf.constant(4.0)

@tf.function
def func(x):
    return x**2 * v

@tf.function
def func2(x):
   return func(x) ** 3 * c

with tf.GradientTape() as tape:
  result = func2(x)
  
tape.gradient(result, [(v, c),()])

[(<tf.Tensor: shape=(), dtype=float32, numpy=24576.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4096.0>),
 <tf.Tensor: shape=(), dtype=float32, numpy=4096.0>]

In [143]:
a = tf.constant([4., 2., 3., 4., 5., 6.])
b = tf.constant([0., 2., 3., 4., 5., 6.])

tf.losses.mean_squared_error(a, b)

<tf.Tensor: shape=(), dtype=float32, numpy=2.6666667>